In [5]:
from pyspark.sql import SparkSession


# Initialize PySpark with MongoDB and Elastic support
spark = (SparkSession.builder.appName("Collecting and Displaying Records")
    # Load support for MongoDB and Elasticsearch
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,org.elasticsearch:elasticsearch-spark-30_2.12:7.14.2")
    # Add Configuration for MongopDB
    .config("spark.mongodb.input.uri", "mongodb://mongo:27017/test.coll")
    .config("spark.mongodb.output.uri", "mongodb://mongo:27017/test.coll")
    .getOrCreate())
sc = spark.sparkContext
sc.setLogLevel("ERROR")

print("\nPySpark initialized...")


PySpark initialized...


In [6]:
on_time_df = spark.read.csv('data/On_Time_On_Time_Performance_2015.csv.bz2', header=True)
on_time_df.limit(20).toPandas()

/Users/kelly/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,_c109
0,2015,1,1,1,4,2015-01-01,AA,19805,AA,N001AA,...,None,None,None,None,None,None,None,None,None,None
1,2015,1,1,1,4,2015-01-01,AA,19805,AA,N001AA,...,None,None,None,None,None,None,None,None,None,None
2,2015,1,1,1,4,2015-01-01,AA,19805,AA,N002AA,...,None,None,None,None,None,None,None,None,None,None
3,2015,1,1,1,4,2015-01-01,AA,19805,AA,N003AA,...,None,None,None,None,None,None,None,None,None,None
4,2015,1,1,1,4,2015-01-01,AA,19805,AA,N003AA,...,None,None,None,None,None,None,None,None,None,None
5,2015,1,1,1,4,2015-01-01,AA,19805,AA,N003AA,...,None,None,None,None,None,None,None,None,None,None
6,2015,1,1,1,4,2015-01-01,AA,19805,AA,N004AA,...,None,None,None,None,None,None,None,None,None,None
7,2015,1,1,1,4,2015-01-01,AA,19805,AA,N005AA,...,None,None,None,None,None,None,None,None,None,None
8,2015,1,1,1,4,2015-01-01,AA,19805,AA,N005AA,...,None,None,None,None,None,None,None,None,None,None
9,2015,1,1,1,4,2015-01-01,AA,19805,AA,N005AA,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
on_time_df.registerTempTable("raw_on_time_performance")

trimmed_cast_performance = spark.sql("""
SELECT
  Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate,
  Carrier, TailNum, FlightNum,
  Origin, OriginCityName, OriginState,
  Dest, DestCityName, DestState,
  DepTime, cast(DepDelay as float), cast(DepDelayMinutes as int),
  cast(TaxiOut as float), cast(TaxiIn as float),
  WheelsOff, WheelsOn,
  ArrTime, cast(ArrDelay as float), cast(ArrDelayMinutes as float),
  cast(Cancelled as int), cast(Diverted as int),
  cast(ActualElapsedTime as float), cast(AirTime as float),
  cast(Flights as int), cast(Distance as float),
  cast(CarrierDelay as float), cast(WeatherDelay as float), 
  cast(NASDelay as float),
  cast(SecurityDelay as float), 
  cast(LateAircraftDelay as float),
  CRSDepTime, CRSArrTime
FROM
  raw_on_time_performance
""")

